<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_p25/blob/main/professor/numpy/notebooks/tarea_tiempos_numpy.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Tarea — Tiempos con distintas estrategias (Python vs NumPy)

Objetivo: practicar varias formas de programar en Python y comparar su rendimiento con NumPy.

Qué harás:
- Implementar 3 problemas (P1 sencillo, P2 intermedio, P3 un poco más complejo).
- Para cada problema, crear 4 versiones: for, list comprehension, generator (yield/generador), y NumPy vectorizado.
- Medir tiempos con `timeit` de forma justa y compararlos.

Reglas mínimas:
- Verifica primero que todas las versiones producen el mismo resultado lógico (mismo tamaño/forma, mismos valores o valores equivalentes).
- Sé consistente: no mezcles listas y arreglos sin aclarar el formato final esperado.
- Para medir generadores, materializa con `list(...)` en el cronómetro para compararlo con las otras estrategias.
- No copies soluciones externas; escribe tu implementación.

Referencia: `07_Vectorizacion_vs_For_vs_Comprehensions.ipynb`.



In [1]:
# Instalación rápida (si la necesitas)

import numpy as np
import timeit


## Problema 1 — Escalar un vector por una constante

Descripción: dado un arreglo 1D `a` y una constante escalar `c`, produce una salida equivalente a `a * c`.

Requisitos:
- Entrada: `a` (1D), `c` (float/int).
- Salida: misma longitud que `a`, valores escalados por `c`.
- Mantén el tipo de salida consistente entre versiones (lista vs ndarray), o documenta la diferencia.

Implementa 4 versiones:
- for loop (acumula resultados con append)
- list comprehension
- generator (yield o expresión generadora)
- NumPy vectorizado

Datos sugeridos: `a = np.arange(n, dtype=float)`, `c = 2.0`.


In [3]:
# P1 — Stubs (completa las funciones)

def p1_for(a: np.ndarray, c: float):
    """Devuelve a*c usando for y append."""
    for i in range(len(a)):
        a[i] = a[i] *c
    return a


def p1_comp(a: np.ndarray, c: float):
    """Devuelve a*c usando list comprehension."""
    return [elem * c for elem in a]


def p1_gen(a: np.ndarray, c: float):
    """Devuelve (como generador) a*c usando yield o gen expr."""
    return (elem * c for elem in a)


def p1_np(a: np.ndarray, c: float):
    """Devuelve a*c usando NumPy vectorizado."""
    return a * c


In [15]:
# P1 — Harness de tiempos (ajusta n y number)

def time_p1(n=5_000_000, number=5):
    a = np.arange(n, dtype=float)
    c = 2.0
    return (
        timeit.timeit(lambda: p1_for(a, c), number=number),
        timeit.timeit(lambda: p1_comp(a, c), number=number),
        timeit.timeit(lambda: list(p1_gen(a, c)), number=number),
        timeit.timeit(lambda: p1_np(a, c), number=number),
    )

time_p1()  # descomenta para probar


(4.916629987000306, 2.869225113000539, 3.164063895000254, 0.04908248499941692)

## Problema 2 — Suma de vecinos 1D (ventana)

Descripción: dado `a` (1D) y una ventana `k` impar (p. ej. 3), calcular `b[i]` como la suma de los `k` vecinos centrados en `i`.

Requisitos:
- Entrada: `a` (1D), `k` impar ≥ 3.
- Borde: puedes ignorar índices fuera de rango, recortar el resultado o replicar/extender bordes; explica tu elección.
- Salida: 1D; documenta si su longitud cambia por tu manejo de bordes.

Implementa 4 versiones: for, list comprehension, generator, NumPy vectorizado (pistas: slicing con desplazamientos, `np.roll`, o una convolución simple).

Datos sugeridos: `a = np.arange(n, dtype=float)`, `k = 3`. 


In [2]:
# P2 — Stubs (completa las funciones)

def p2_for(a: np.ndarray, k: int = 3):
    """Devuelve suma de vecinos (1D) con for. Manejo de bordes a tu elección."""
    b = []
    for i in range(len(a)):
        if i ==0:
            b.append(a[i] + a[i+1])
        elif i == len(a)-1:
            b.append(a[i] + a[i-1])
        else:
            b.append(a[i-1] + a[i] + a[i+1])
    return b



def p2_comp(a: np.ndarray, k: int = 3):
    """List comprehension."""
    b = [a[i] + a[i+1] if i == 0 else a[i-1] + a[i] if i == len(a)-1 else a[i-1] + a[i] + a[i+1]  for i in range(len(a))]
    return b


def p2_gen(a: np.ndarray, k: int = 3):
    """Generator (yield o gen expr)."""
    return (a[i] + a[i+1] if i == 0 else a[i-1] + a[i] if i == len(a)-1 else a[i-1] + a[i] + a[i+1]  for i in range(len(a)))


def p2_np(a: np.ndarray, k: int = 3):
    """NumPy vectorizado (slicing/roll/convolución simple)."""
    a_r = np.roll(a, 1)
    a_l = np.roll(a, -1)
    return a_l + a + a_r


In [4]:
# P2 — Harness de tiempos

def time_p2(n=500_000, number=3):
    a = np.arange(n, dtype=float)
    k = 3
    return (
        timeit.timeit(lambda: p2_for(a, k), number=number),
        timeit.timeit(lambda: p2_comp(a, k), number=number),
        timeit.timeit(lambda: list(p2_gen(a, k)), number=number),
        timeit.timeit(lambda: p2_np(a, k), number=number),
    )

time_p2()  # descomenta para probar


(0.3196885850002218,
 0.30828618900022775,
 0.33248849500023425,
 0.014184820000082254)

## Problema 3 — Transformación no lineal y filtrado

Descripción: dado `a` (1D float), aplica una transformación no lineal y filtra con un umbral.

Requisitos:
- Transformación propuesta (de ejemplo): `np.sin(a) + a**2`.
- Entrada: `a` (1D float), `umbral` (float).
- Salida: colección con los elementos resultantes que superan `umbral`.
- Mantén clara la diferencia entre devolver lista vs ndarray.

Implementa 4 versiones: for, list comprehension, generator, NumPy vectorizado (ufuncs + máscara booleana).

Datos sugeridos: `a = np.linspace(0, 1000, n)`, `umbral = 10.0`. 


In [15]:
# P3 — Stubs (completa las funciones)

def p3_for(a: np.ndarray, umbral: float):
    """Filtra tras transformación no lineal con for."""
    b = []
    res = []
    for elem in a:
        b.append(np.cos(elem) - elem//2)
    for elem in b:
        if elem > umbral:
            res.append(elem)
    return res


def p3_comp(a: np.ndarray, umbral: float):
    """List comprehension."""
    b = [np.cos(elem) - elem//2 for elem in a]
    res = [elem for elem in b if elem > umbral]
    return res


def p3_gen(a: np.ndarray, umbral: float):
    """Generator (yield o gen expr)."""
    return (np.cos(elem) - elem//2 for elem in a if np.cos(elem) - elem//2 > umbral)


def p3_np(a: np.ndarray, umbral: float):
    """NumPy vectorizado (ufuncs + máscara booleana)."""
    transformado = np.cos(a) - a//2
    mask = transformado > umbral
    return transformado[mask]

In [25]:
# P3 — Harness de tiempos

def time_p3(n=5_000_000, number=3):
    a = np.linspace(0, 1000, n, dtype=float)
    umbral = 10.0
    return (
        timeit.timeit(lambda: p3_for(a, umbral), number=number),
        timeit.timeit(lambda: p3_comp(a, umbral), number=number),
        timeit.timeit(lambda: list(p3_gen(a, umbral)), number=number),
        timeit.timeit(lambda: p3_np(a, umbral), number=number),
    )

time_p3()  # descomenta para probar


(7.670921963999717, 7.551161205999961, 7.3035017250003875, 0.26664458499999455)

## Guía de medición y reporte

- Usa `timeit.timeit` con el mismo `number` de repeticiones para todas las versiones.
- Antes de medir, ejecuta cada función una vez (warm‑up) si tu entorno lo requiere.
- Reporta tiempos en una tabla simple o tupla por problema: `(for, comp, gen, numpy)`.
- Interpreta resultados: ¿qué versión gana?, ¿por cuánto?, ¿cambia con `n`?
- Evita medir al mismo tiempo código que imprime o muestra gráficos.

Sugerencia: prueba varios tamaños `n` (por ejemplo: 10^4, 10^5, 10^6) y observa tendencias.


##P1 tabla con number = 5
|  n  |tiempo for         |list comprehension |generator           |numpy                 |
|-----|-------------------|-------------------|--------------------|----------------------|
| 100k|0.14267290600037086|0.05218020699976478|0.057727882000108366|0.00036060100046597654|
| 200k|0.2725760140001512 |0.10785882800064428|0.11782145899996976 |0.0007165829993027728 |
| 500k|0.5404342669999096 |0.27413474900004076|0.3013467840000885  |0.0028762550000465126 |
| 800k|0.807676698999785  |0.43916778299990256|0.48380014399936044 |0.004117022000173165  |
| 1M  |1.0604532090001157 |0.5480016349993093 |0.602993967000657   |0.0060772119995817775 |
| 5M  |4.916629987000306  |2.869225113000539  |3.164063895000254   |0.04908248499941692   |

La version que usa numpy siempre funciona de manera mas eficiente/rapida a comparacion de las demas mientras que la peor en terminos de tiempo es usar el for. Sin embargo, cuando se usa el for a traves de un list comprehension es mucho mas rapido, hasta es un poco mejor que los generadores.

##P2 con number = 5
|  n  |tiempo for         |list comprehension |generator           |numpy                 |
|-----|-------------------|-------------------|--------------------|----------------------|
| 100k|0.06722699700003432|0.0632214790002763 | 0.06541336200007208|0.002145092000318982  |
| 200k|0.13390825700025744|0.12694243300029484|0.13549481900008686 |0.005069948999334883  |
| 500k|0.3317850160001399 |0.3316779440001483 |0.36666671100010717 |0.015507285999774467  |
| 800k|0.5333624340000824 |0.5250538700001925 |0.5379247820001183  |0.01817423900047288   |
| 1M  |0.6563738089998878 |0.5498311510000349 |0.5411917299998095  |0.02167522599984295   |
| 5M  |3.3337465719996544 |3.278671379000116  |3.39461583100001    |0.0834486020003169    |

Ocurrio de nuevo que numpy es mucho mas rapido que todo lo demas. Sin embargo, en este caso las variaciones entre los 3 metodos son minimas ya que se tardan los otros 3 tiempos relativamente similares.

##P3 con number=5
|  n  |tiempo for         |list comprehension |generator           |numpy                 |
|-----|-------------------|-------------------|--------------------|----------------------|
| 100k|0.156739025999741  |0.15139056400039408|0.14802031300041563 |0.00492725500043889   |
| 200k|0.29519995499958895|0.29792902499957563|0.2948185029999877  |0.009876176999568997  |
| 500k|0.7606359940000402 |0.7553392419995362 |0.7401742860001832  |0.024889985999834607  |
| 800k|1.2106405539998377 |1.2007762970006297 |1.193161426000188   |0.04439699999966251   |
| 1M  |1.5081692089997887 |1.5038073870000517 |1.4680889549999847  |0.05142907200024638   |
| 5M  |7.670921963999717  |7.551161205999961  |7.3035017250003875  |0.26664458499999455   |

Si se busca mantener un meta saludable y variado, numpy necesita un nerf pronto. Pero fuera de metaforas, numpy es lo mas rapido y ni tiene competencia con las demas maneras de hacerlo. Aunque luego list comprehensions y generators van alternando entre cual es mas rapida. Lo cual es algo interesante porque al parecer no siguen alguna regla fija y parece aleatorio debido a lo cerca que son sus tiempos.